# Symmetry reduction

**Adapted from**: [SymbolicWedderburn example](https://github.com/kalmarek/SymbolicWedderburn.jl/blob/tw/ex_sos/examples/ex_C4.jl)

In [1]:
import MutableArithmetics
const MA = MutableArithmetics
using MultivariatePolynomials
const MP = MultivariatePolynomials
using MultivariateBases
const MB = MultivariateBases

using DynamicPolynomials
@polyvar x[1:4]

(DynamicPolynomials.PolyVar{true}[x₁, x₂, x₃, x₄],)

We would like to find the minimum value of the polynomial

In [2]:
poly = sum(x) + sum(x.^2)

x₁² + x₂² + x₃² + x₄² + x₁ + x₂ + x₃ + x₄

As we can decouple the problem for each `x[i]` for which `x[i] + x[i]^2` has
minimum value 0.25, we would expect to get `-1` as answer.
Can this decoupling be exploited by SumOfSquares as well ?
For this, we need to use a certificate that can exploit the permutation symmetry of the polynomial.

In [3]:
using SumOfSquares

We define the symmetry group as a permutation group in the variables.
In order to do that, we define the action of a permutation on a monomial
as the monomial obtained after permuting the variables.

In [4]:
using PermutationGroups
G = PermGroup([perm"(1,2,3,4)"])

Permutation group on 1 generator generated by
 (1,2,3,4)

We can use this certificate as follows:

In [5]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
pattern = Symmetry.Pattern(G, Symmetry.VariablePermutation())
con_ref = @constraint(model, poly - t in SOSCone(), symmetry = pattern)
optimize!(model)
value(t)

R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x₁ + x₂ + x₃ + x₄, ( 1.0*E(1)^0)]
m = 2
d = 1
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x₁ + -x₃, x₂ + -x₄]
m = 1
d = 2
R * basis.monomials = DynamicPolynomials.Polynomial{true, Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}[x₁ + -x₂ + x₃ + -x₄]
m = 1
d = 1
Iter: 11 Ap: 9.00e-01 Pobj: -1.0000000e+00 Ad: 1.00e+00 Dobj: -1.0000000e+00 
Success: SDP solved
Primal objective value: -1.0000000e+00 
Dual objective value: -1.0000000e+00 
Relative primal infeasibility: 8.79e-14 
Relative dual infeasibility: 1.07e-10 
Real Relative Gap: 1.32e-09 
XZ Relative Gap: 1.52e-09 
DIMACS error measures: 9.48e-14 0.00e+00 2.21e-10 0.00e+00 1.32e-09 1.52e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 

2×5 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

2×5 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 1.0  0.0  -1.0   0.0  0.0
 0.0  1.0   0.0  -1.0  0.0

1×5 Matrix{Cyclotomics.Cyclotomic{Float64, SparseArrays.SparseVector{Float64, Int64}}}:
 1.0  -1.0  1.0  -1.0  0.0

-1.0000000002081693

We indeed find `-1`, let's verify that symmetry was exploited:

In [6]:
for g in gram_matrix(con_ref).sub_gram_matrices
    println(g.basis.polynomials)
end

DynamicPolynomials.Polynomial{true, Float64}[x₁ + x₂ + x₃ + x₄, 1.0]
DynamicPolynomials.Polynomial{true, Float64}[x₁ - x₃]
DynamicPolynomials.Polynomial{true, Float64}[x₂ - x₄]
DynamicPolynomials.Polynomial{true, Float64}[x₁ - x₂ + x₃ - x₄]


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*